# Tutorial 06, case 7b: Stokes problem with Neumann control

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Gamma_{obs}} (v - v_d)^2 dx + \frac{\alpha_1}{2} \int_{\Gamma_C} |\nabla_{\mathbf{t}} u|^2 ds + \frac{\alpha_2}{2} \int_{\Gamma_C} |u|^2 ds$$
s.t.
$$\begin{cases}
- \nu \Delta v + \nabla p = f       & \text{in } \Omega\\
             \text{div} v = 0       & \text{in } \Omega\\
                        v = g       & \text{on } \Gamma_{in}\\
                        v = 0       & \text{on } \Gamma_{w}\\
   p n - \nu \partial_n v = u       & \text{on } \Gamma_{C}
\end{cases}$$

where
$$\begin{align*}
& \Omega                      & \text{unit square}\\
& \Gamma_{in}                 & \text{has boundary id 1}\\
& \Gamma_{w}                  & \text{has boundary id 2}\\
& \Gamma_{C}                  & \text{has boundary id 3}\\
& \Gamma_{obs}                & \text{has boundary id 4}\\
& u \in [L^2(\Gamma_C)]^2     & \text{control variable}\\
& v \in [H^1(\Omega)]^2       & \text{state velocity variable}\\
& p \in L^2(\Omega)           & \text{state pressure variable}\\
& \alpha_1, \alpha_2 > 0      & \text{penalization parameters}\\
& v_d                         & \text{desired state}\\
& f                           & \text{forcing term}\\
& g                           & \text{inlet profile}\\
\end{align*}$$
using an adjoint formulation solved by a one shot approach

In [ ]:
import dolfinx.fem
import dolfinx.io
import dolfinx.mesh
import mpi4py
import numpy as np
import numpy.typing as npt
import petsc4py
import ufl

In [ ]:
import multiphenicsx.fem
import multiphenicsx.io

### Mesh

In [ ]:
if mpi4py.MPI.COMM_WORLD.size > 1:
    mesh_ghost_mode = dolfinx.mesh.GhostMode.shared_facet  # shared_facet ghost mode is required by dS
else:
    mesh_ghost_mode = dolfinx.mesh.GhostMode.none
with dolfinx.io.XDMFFile(mpi4py.MPI.COMM_WORLD, "data/bifurcation.xdmf", "r") as infile:
    mesh = infile.read_mesh(mesh_ghost_mode)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    boundaries = infile.read_meshtags(mesh, name="boundaries")
boundaries_1 = boundaries.indices[boundaries.values == 1]
boundaries_2 = boundaries.indices[boundaries.values == 2]
boundaries_3 = boundaries.indices[boundaries.values == 3]
boundaries_12 = boundaries.indices[np.isin(boundaries.values, (1, 2))]

In [ ]:
# Define associated measures
ds = ufl.Measure("ds")(subdomain_data=boundaries)
dS = ufl.Measure("dS")(subdomain_data=boundaries)

In [ ]:
# Normal and tangent
n = ufl.FacetNormal(mesh)
t = ufl.as_vector([n[1], -n[0]])

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

### Function spaces

In [ ]:
Y_velocity = dolfinx.fem.VectorFunctionSpace(mesh, ("Lagrange", 2))
Y_pressure = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
U = dolfinx.fem.VectorFunctionSpace(mesh, ("Lagrange", 2))
Q_velocity = Y_velocity.clone()
Q_pressure = Y_pressure.clone()

### Restrictions

In [ ]:
dofs_Y_velocity = np.arange(0, Y_velocity.dofmap.index_map.size_local + Y_velocity.dofmap.index_map.num_ghosts)
dofs_Y_pressure = np.arange(0, Y_pressure.dofmap.index_map.size_local + Y_pressure.dofmap.index_map.num_ghosts)
dofs_U = dolfinx.fem.locate_dofs_topological(U, boundaries.dim, boundaries_3)
dofs_Q_velocity = dofs_Y_velocity
dofs_Q_pressure = dofs_Y_pressure
restriction_Y_velocity = multiphenicsx.fem.DofMapRestriction(Y_velocity.dofmap, dofs_Y_velocity)
restriction_Y_pressure = multiphenicsx.fem.DofMapRestriction(Y_pressure.dofmap, dofs_Y_pressure)
restriction_U = multiphenicsx.fem.DofMapRestriction(U.dofmap, dofs_U)
restriction_Q_velocity = multiphenicsx.fem.DofMapRestriction(Q_velocity.dofmap, dofs_Q_velocity)
restriction_Q_pressure = multiphenicsx.fem.DofMapRestriction(Q_pressure.dofmap, dofs_Q_pressure)
restriction = [
    restriction_Y_velocity, restriction_Y_pressure, restriction_U, restriction_Q_velocity, restriction_Q_pressure]

### Trial and test functions

In [ ]:
(v, p) = (ufl.TrialFunction(Y_velocity), ufl.TrialFunction(Y_pressure))
(w, q) = (ufl.TestFunction(Y_velocity), ufl.TestFunction(Y_pressure))
u = ufl.TrialFunction(U)
r = ufl.TestFunction(U)
(z, b) = (ufl.TrialFunction(Q_velocity), ufl.TrialFunction(Q_pressure))
(s, d) = (ufl.TestFunction(Q_velocity), ufl.TestFunction(Q_pressure))

 ### Problem data

In [ ]:
nu = 0.04
alpha_1 = 0.001
alpha_2 = 0.1 * alpha_1
x = ufl.SpatialCoordinate(mesh)
a = 1.0
c = 0.8
v_d = ufl.as_vector((
    a * (c * 10.0 * (x[1]**3 - x[1]**2 - x[1] + 1.0)) + ((1.0 - c) * 10.0 * (-x[1]**3 - x[1]**2 + x[1] + 1.0)),
    0.0))
ff = dolfinx.fem.Constant(mesh, tuple(petsc4py.PETSc.ScalarType(0) for _ in range(2)))


def g_eval(x: npt.NDArray[np.float64]) -> npt.NDArray[petsc4py.PETSc.ScalarType]:
    """Return the parabolic velocity profile at the inlet."""
    values = np.zeros((2, x.shape[1]))
    values[0, :] = 10.0 * a * (x[1, :] + 1.0) * (1.0 - x[1, :])
    return values


g = dolfinx.fem.Function(Y_velocity)
g.interpolate(g_eval)
bc0 = dolfinx.fem.Function(Y_velocity)

### Optimality conditions

In [ ]:
def tracking(v: ufl.Argument, w: ufl.Argument) -> ufl.core.expr.Expr:
    """Return the UFL expression corresponding to the tracking term."""
    return ufl.inner(v, w)("-")


def penalty(u: ufl.Argument, r: ufl.Argument) -> ufl.core.expr.Expr:
    """Return the UFL expression corresponding to the penalty term."""
    return alpha_1 * ufl.inner(ufl.grad(u) * t, ufl.grad(r) * t) + alpha_2 * ufl.inner(u, r)


a = [[tracking(v, w) * dS(4), None, None, nu * ufl.inner(ufl.grad(z), ufl.grad(w)) * ufl.dx,
      - ufl.inner(b, ufl.div(w)) * ufl.dx],
     [None, None, None, - ufl.inner(ufl.div(z), q) * ufl.dx, None],
     [None, None, penalty(u, r) * ds(3), - ufl.inner(z, r) * ds(3), None],
     [nu * ufl.inner(ufl.grad(v), ufl.grad(s)) * ufl.dx, - ufl.inner(p, ufl.div(s)) * ufl.dx,
      - ufl.inner(u, s) * ds(3), None, None],
     [- ufl.inner(ufl.div(v), d) * ufl.dx, None, None, None, None]]
f = [tracking(v_d, w) * dS(4),
     None,
     None,
     ufl.inner(ff, s) * ufl.dx,
     None]
a[0][0] += dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0)) * ufl.inner(v, w) * (ds(1) + ds(2))
a[3][3] = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0)) * ufl.inner(z, s) * (ds(1) + ds(2))
f[1] = ufl.inner(dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0)), q) * ufl.dx
f[2] = ufl.inner(dolfinx.fem.Constant(mesh, tuple(petsc4py.PETSc.ScalarType(0) for _ in range(2))), r) * ufl.dx
f[4] = ufl.inner(dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0)), d) * ufl.dx
bdofs_Y_velocity_1 = dolfinx.fem.locate_dofs_topological(
    (Y_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_1)
bdofs_Y_velocity_2 = dolfinx.fem.locate_dofs_topological(
    (Y_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_2)
bdofs_Q_velocity_12 = dolfinx.fem.locate_dofs_topological(
    (Q_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_12)
bc = [dolfinx.fem.DirichletBC(g, bdofs_Y_velocity_1, Y_velocity),
      dolfinx.fem.DirichletBC(bc0, bdofs_Y_velocity_2, Y_velocity),
      dolfinx.fem.DirichletBC(bc0, bdofs_Q_velocity_12, Q_velocity)]

### Solution

In [ ]:
(v, p) = (dolfinx.fem.Function(Y_velocity), dolfinx.fem.Function(Y_pressure))
u = dolfinx.fem.Function(U)
(z, b) = (dolfinx.fem.Function(Q_velocity), dolfinx.fem.Function(Q_pressure))

### Cost functional

In [ ]:
J = 0.5 * tracking(v - v_d, v - v_d) * dS(4) + 0.5 * penalty(u, u) * ds(3)

### Uncontrolled functional value

In [ ]:
# Extract state forms from the optimality conditions
a_state = [[ufl.replace(a[i][j], {s: w, d: q}) if a[i][j] is not None else None for j in (0, 1)] for i in (3, 4)]
f_state = [ufl.replace(f[i], {s: w, d: q}) for i in (3, 4)]
bc_state = [bc[0], bc[1]]

In [ ]:
# Assemble the block linear system for the state
A_state = multiphenicsx.fem.assemble_matrix_block(
    a_state, bcs=bc_state, restriction=([restriction[i] for i in (3, 4)], [restriction[j] for j in (0, 1)]))
A_state.assemble()
F_state = multiphenicsx.fem.assemble_vector_block(
    f_state, a_state, bcs=bc_state, restriction=[restriction[i] for i in (3, 4)])

In [ ]:
# Solve
vp = multiphenicsx.fem.create_vector_block([f[j] for j in (0, 1)], restriction=[restriction[j] for j in (0, 1)])
ksp = petsc4py.PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A_state)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F_state, vp)
vp.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with multiphenicsx.fem.BlockVecSubVectorWrapper(vp, [c.function_space.dofmap for c in (v, p)]) as vp_wrapper:
    for vp_wrapper_local, component in zip(vp_wrapper, (v, p)):
        with component.vector.localForm() as component_local:
            component_local[:] = vp_wrapper_local

In [ ]:
J_uncontrolled = mesh.comm.allreduce(dolfinx.fem.assemble_scalar(J), op=mpi4py.MPI.SUM)
print("Uncontrolled J =", J_uncontrolled)
assert np.isclose(J_uncontrolled, 2.8479865)

In [ ]:
multiphenicsx.io.plot_vector_field(v, "uncontrolled state velocity", glyph_factor=1e-2)

In [ ]:
multiphenicsx.io.plot_scalar_field(p, "uncontrolled state pressure")

### Optimal control

In [ ]:
# Assemble the block linear system for the optimality conditions
A = multiphenicsx.fem.assemble_matrix_block(a, bcs=bc, restriction=(restriction, restriction))
A.assemble()
F = multiphenicsx.fem.assemble_vector_block(f, a, bcs=bc, restriction=restriction)

In [ ]:
# Solve
vpuzb = multiphenicsx.fem.create_vector_block(f, restriction=restriction)
ksp = petsc4py.PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, vpuzb)
vpuzb.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with multiphenicsx.fem.BlockVecSubVectorWrapper(
        vpuzb, [c.function_space.dofmap for c in (v, p, u, z, b)], restriction) as vpuzb_wrapper:
    for vpuzb_wrapper_local, component in zip(vpuzb_wrapper, (v, p, u, z, b)):
        with component.vector.localForm() as component_local:
            component_local[:] = vpuzb_wrapper_local

In [ ]:
J_controlled = mesh.comm.allreduce(dolfinx.fem.assemble_scalar(J), op=mpi4py.MPI.SUM)
print("Optimal J =", J_controlled)
assert np.isclose(J_controlled, 1.7643950)

In [ ]:
multiphenicsx.io.plot_vector_field(v, "state velocity", glyph_factor=1e-2)

In [ ]:
multiphenicsx.io.plot_scalar_field(p, "state pressure")

In [ ]:
multiphenicsx.io.plot_vector_field(u, "control", glyph_factor=1e-1)

In [ ]:
multiphenicsx.io.plot_vector_field(z, "adjoint velocity", glyph_factor=1e-1)

In [ ]:
multiphenicsx.io.plot_scalar_field(b, "adjoint pressure")